In [1]:
!pip install nemo_toolkit['all']
!pip install pynini
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 39.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 KB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 KB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━

In [4]:
# Getting Accelerator  ..
try:
  import os
  import torch
  import soundfile as sf
  from googletrans import Translator
  translator = Translator()
  print('Basic modules has been imported')
  
except:
  print('There are some issues with the import, Please install the basic modules and try again ..')

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
# Load FastPitch
try:
  from nemo.collections.tts.models import FastPitchModel
  spec_generator = FastPitchModel.from_pretrained("nvidia/tts_en_fastpitch")
  print('FastPitch model has been imported successfully  ..')
except:
  print('There is some problem with importing the model ..')

# Load vocoder
try:
  from nemo.collections.tts.models import HifiGanModel
  model = HifiGanModel.from_pretrained(model_name="nvidia/tts_hifigan")
  print('Vocoder has been imported too ..')
except:
  print('Vocoder is not getting imported  ..')
  print('Congratts!! All modules has been imported successfully  ..')

##  Getting the Text inputs for TTS  ..
def get_input(input_mode):
  try:
    # Moving with Sample Text
    if input_mode=='sample text':
      input_text = "Text-speak is especially popular among the digital natives or net-generation."
      print(input_text)
      print('Please wait we are processing your inputs ..')

    elif input_mode=='custom text':
      input_text = input('Please type your words .. ')
      print(' Please wait we are processing your inputs ..')

    elif input_mode=='file':
      print('Okk Great! You need to provide a txt file here. I can only process a txt file')
      file_path = input('Please give the path of file to be processed .. ')
      file1 = open(file_path, "r" , encoding="utf8")
      file = file1.readlines()
      print(file) 
      file1.close()
      print('Okk great .. Please hold on, we are processing your text.')
  except:
    print('Please provide correct file path ..')
  return input_text

# TTS Synthesis with model ..
def get_speech(text):
  try:
    parsed = spec_generator.parse(text)
    spectrogram = spec_generator.generate_spectrogram(tokens=parsed)
    audio = model.convert_spectrogram_to_audio(spec=spectrogram)
    print('Congratts Your inputted Text has been processed ..')

    # Save the audio to disk in a file called speech.wav
    sf.write("speech.wav", audio.to('cpu').detach().numpy()[0], 22050)
    print ('Text-to-Speech Synthesis has been done and output file has been saved successfully ..')
    print('Please check "speech.wav" file .')
  except:
        print('There are some problems with TTS ..')

def get_hindi_speech(text):
  pass

input_mode = input(' Please choose an option - "sample text, custom text, file" ')
if input_mode=="sample text" or "custom text" or "file":
  text = get_input(input_mode)
  if translator.detect(text).lang=='en':
    get_speech(text)
  elif translator.detect('This sentence is written in English.').lang=='hi':
    get_hindi_speech(text)
  
else:
  print('There is something wrong with your selection. Please choose a correct option')

[NeMo I 2023-02-04 05:53:51 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2023-02-04 05:54:19 experimental:27] Module <class 'nemo_text_processing.g2p.modules.IPAG2P'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-02-04 05:54:20 modules:95] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2023-02-04 05:54:20 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_m

[NeMo I 2023-02-04 05:54:21 features:267] PADDING: 1
[NeMo I 2023-02-04 05:54:22 save_restore_connector:243] Model FastPitchModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--tts_en_fastpitch/snapshots/2c8305b7b41b33fd6367f0635796dc3a7a33cbf9/tts_en_fastpitch.nemo.


[NeMo W 2023-02-04 05:54:38 modelPT:156] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2023-02-04 05:54:38 modelPT:163] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2023-02-04 05:54:38 features:267] PADDING: 0


[NeMo W 2023-02-04 05:54:38 features:244] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2023-02-04 05:54:38 features:267] PADDING: 0
[NeMo I 2023-02-04 05:54:39 save_restore_connector:243] Model HifiGanModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--tts_hifigan/snapshots/3ba1fed954276287015654bf4c78060ffc9a4772/tts_hifigan.nemo.


[NeMo W 2023-02-04 06:00:33 fastpitch:260] parse() is meant to be called in eval mode.
[NeMo W 2023-02-04 06:00:33 fastpitch:325] generate_spectrogram() is meant to be called in eval mode.
